In [2]:
# Import python packages
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

# Check version numbers
print(f'numpy: {np.version.version}')
print(f'pandas: {pd.__version__}')
print(f'scipy: {sp.__version__}')
print(f'scikit-learn: {sk.__version__}')
print(f'seaborn: {sns.__version__}')

numpy: 2.3.2
pandas: 2.3.1
scipy: 1.16.1
scikit-learn: 1.7.1
seaborn: 0.13.2


In [3]:
df = pd.read_csv("/workspaces/4Geeks_final_project/data/superstore_final_dataset (1).csv", encoding='latin1')

df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold N Roll Cart System,22.3680


In [4]:
import regex as re

def preprocess_text(text):
    text = str(text).strip()
    
    text = re.sub(r"<.*?>", " ", text)
    
    text = re.sub(r"\s+", " ", text)
    
    # (keep / and - for dates and codes)
    text = re.sub(r"[^a-zA-Z0-9\s\-/]", "", text)
    
    # Lowercase for consistency
    text = text.lower().strip()
    
    return text

cols_to_clean = ["Order_Date", "Ship_Date","Ship_Mode", "Customer_Name", "Segment", 
                 "Country", "City", "State", "Region",
                 "Category", "Sub_Category", "Product_Name"]

for col in cols_to_clean:
    df[col] = df[col].astype(str).apply(preprocess_text)


#df[["Order_Date", "Ship_Date","Customer_Name", "Segment", "Country", "City", "State", "Region", "Category", "Sub_Category", "Product_Name"]] = df[["Order_Date", "Ship_Date","Customer_Name", "Segment", "Country", "City", "State", "Region", "Category", "Sub_Category", "Product_Name"]].apply(preprocess_text)
df.head(20)

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,second class,CG-12520,claire gute,consumer,united states,henderson,kentucky,42420.0,south,FUR-BO-10001798,furniture,bookcases,bush somerset collection bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,second class,CG-12520,claire gute,consumer,united states,henderson,kentucky,42420.0,south,FUR-CH-10000454,furniture,chairs,hon deluxe fabric upholstered stacking chairs ...,731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,second class,DV-13045,darrin van huff,corporate,united states,los angeles,california,90036.0,west,OFF-LA-10000240,office supplies,labels,self-adhesive address labels for typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,standard class,SO-20335,sean o donnel,consumer,united states,fort lauderdale,florida,33311.0,south,FUR-TA-10000577,furniture,tables,bretford cr4500 series slim rectangular table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,standard class,SO-20335,sean o donnel,consumer,united states,fort lauderdale,florida,33311.0,south,OFF-ST-10000760,office supplies,storage,eldon fold n roll cart system,22.3680
5,6,CA-2015-115812,9/6/2015,14/06/2015,standard class,BH-11710,brosina hoffman,consumer,united states,los angeles,california,90032.0,west,FUR-FU-10001487,furniture,furnishings,eldon expressions wood and plastic desk access...,48.8600
6,7,CA-2015-115812,9/6/2015,14/06/2015,standard class,BH-11710,brosina hoffman,consumer,united states,los angeles,california,90032.0,west,OFF-AR-10002833,office supplies,art,newell 322,7.2800
7,8,CA-2015-115812,9/6/2015,14/06/2015,standard class,BH-11710,brosina hoffman,consumer,united states,los angeles,california,90032.0,west,TEC-PH-10002275,technology,phones,mitel 5320 ip phone voip phone,907.1520
8,9,CA-2015-115812,9/6/2015,14/06/2015,standard class,BH-11710,brosina hoffman,consumer,united states,los angeles,california,90032.0,west,OFF-BI-10003910,office supplies,binders,dxl angle-view binders with locking rings by s...,18.5040
9,10,CA-2015-115812,9/6/2015,14/06/2015,standard class,BH-11710,brosina hoffman,consumer,united states,los angeles,california,90032.0,west,OFF-AP-10002892,office supplies,appliances,belkin f5c206vtel 6 outlet surge,114.9000


In [5]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'], dayfirst=True, errors='coerce')
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'], dayfirst=True, errors='coerce')


df['Order_Date'] = df['Order_Date'].astype("int64") / 1e9
df['Ship_Date'] = df['Ship_Date'].astype("int64") / 1e9

df["Turnaround_Time"] = df["Order_Date"] - df["Ship_Date"]

df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Turnaround_Time
0,1,CA-2017-152156,1.510099e+09,1.510358e+09,second class,CG-12520,claire gute,consumer,united states,henderson,kentucky,42420.0,south,FUR-BO-10001798,furniture,bookcases,bush somerset collection bookcase,261.9600,-259200.0
1,2,CA-2017-152156,1.510099e+09,1.510358e+09,second class,CG-12520,claire gute,consumer,united states,henderson,kentucky,42420.0,south,FUR-CH-10000454,furniture,chairs,hon deluxe fabric upholstered stacking chairs ...,731.9400,-259200.0
2,3,CA-2017-138688,1.497226e+09,1.497571e+09,second class,DV-13045,darrin van huff,corporate,united states,los angeles,california,90036.0,west,OFF-LA-10000240,office supplies,labels,self-adhesive address labels for typewriters b...,14.6200,-345600.0
3,4,US-2016-108966,1.476144e+09,1.476749e+09,standard class,SO-20335,sean o donnel,consumer,united states,fort lauderdale,florida,33311.0,south,FUR-TA-10000577,furniture,tables,bretford cr4500 series slim rectangular table,957.5775,-604800.0
4,5,US-2016-108966,1.476144e+09,1.476749e+09,standard class,SO-20335,sean o donnel,consumer,united states,fort lauderdale,florida,33311.0,south,OFF-ST-10000760,office supplies,storage,eldon fold n roll cart system,22.3680,-604800.0


In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
df['Customer_Name_encoded'] = le.fit_transform(df['Customer_Name'])
df['City_encoded'] = le.fit_transform(df['City'])
df['State_encoded'] = le.fit_transform(df['State'])
df['Postal_Code_encoded'] = le.fit_transform(df['Postal_Code'])

# One-hot encode low-cardinality categorical columns
df = pd.get_dummies(df, columns=['Ship_Mode', 'Segment', 'Country', 'Region', 'Category', 'Sub_Category'])

df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Customer_ID,Customer_Name,City,State,Postal_Code,Product_ID,...,Sub_Category_envelopes,Sub_Category_fasteners,Sub_Category_furnishings,Sub_Category_labels,Sub_Category_machines,Sub_Category_paper,Sub_Category_phones,Sub_Category_storage,Sub_Category_supplies,Sub_Category_tables
0,1,CA-2017-152156,1.510099e+09,1.510358e+09,CG-12520,claire gute,henderson,kentucky,42420.0,FUR-BO-10001798,...,False,False,False,False,False,False,False,False,False,False
1,2,CA-2017-152156,1.510099e+09,1.510358e+09,CG-12520,claire gute,henderson,kentucky,42420.0,FUR-CH-10000454,...,False,False,False,False,False,False,False,False,False,False
2,3,CA-2017-138688,1.497226e+09,1.497571e+09,DV-13045,darrin van huff,los angeles,california,90036.0,OFF-LA-10000240,...,False,False,False,True,False,False,False,False,False,False
3,4,US-2016-108966,1.476144e+09,1.476749e+09,SO-20335,sean o donnel,fort lauderdale,florida,33311.0,FUR-TA-10000577,...,False,False,False,False,False,False,False,False,False,True
4,5,US-2016-108966,1.476144e+09,1.476749e+09,SO-20335,sean o donnel,fort lauderdale,florida,33311.0,OFF-ST-10000760,...,False,False,False,False,False,False,False,True,False,False


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=100)
product_tfidf = tfidf.fit_transform(df['Product_Name'].fillna(''))
product_tfidf_df = pd.DataFrame(product_tfidf.toarray(), columns=tfidf.get_feature_names_out())
df = pd.concat([df.reset_index(drop=True), product_tfidf_df.reset_index(drop=True)], axis=1)

df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Customer_ID,Customer_Name,City,State,Postal_Code,Product_ID,...,task,usb,wall,white,wilson,wire,wireless,with,wood,xerox
0,1,CA-2017-152156,1.510099e+09,1.510358e+09,CG-12520,claire gute,henderson,kentucky,42420.0,FUR-BO-10001798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,CA-2017-152156,1.510099e+09,1.510358e+09,CG-12520,claire gute,henderson,kentucky,42420.0,FUR-CH-10000454,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,CA-2017-138688,1.497226e+09,1.497571e+09,DV-13045,darrin van huff,los angeles,california,90036.0,OFF-LA-10000240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,US-2016-108966,1.476144e+09,1.476749e+09,SO-20335,sean o donnel,fort lauderdale,florida,33311.0,FUR-TA-10000577,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,US-2016-108966,1.476144e+09,1.476749e+09,SO-20335,sean o donnel,fort lauderdale,florida,33311.0,OFF-ST-10000760,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Columns: 149 entries, Row_ID to xerox
dtypes: bool(32), float64(105), int64(5), object(7)
memory usage: 9.0+ MB


In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Sales", "Order_ID", "Customer_ID", "Product_ID","Customer_Name","Product_Name", "City", "State", "Postal_Code", "Ship_Date"]) 
y = df["Sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, train_size = 0.80)

In [10]:
X

,Row_ID,Order_Date,Turnaround_Time,Customer_Name_encoded,City_encoded,State_encoded,Postal_Code_encoded,Ship_Mode_first class,Ship_Mode_same day,Ship_Mode_second class,...,task,usb,wall,white,wilson,wire,wireless,with,wood,xerox
0,1,1.510099e+09,-259200.0,166,194,15,215,False,False,True,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.510099e+09,-259200.0,166,194,15,215,False,False,True,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.497226e+09,-345600.0,201,265,3,513,False,False,True,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1.476144e+09,-604800.0,687,153,8,168,False,False,False,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1.476144e+09,-604800.0,687,153,8,168,False,False,False,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,1.495325e+09,-604800.0,663,80,11,343,False,False,False,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9796,9797,1.452557e+09,-432000.0,164,483,33,223,False,False,False,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9797,9798,1.452557e+09,-432000.0,164,483,33,223,False,False,False,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9798,9799,1.452557e+09,-432000.0,164,483,33,223,False,False,False,...,0.0,0.640049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y

0       261.9600
1       731.9400
2        14.6200
3       957.5775
4        22.3680
          ...   
9795      3.7980
9796     10.3680
9797    235.1880
9798     26.3760
9799     10.3840
Name: Sales, Length: 9800, dtype: float64

In [12]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [13]:
print(f"Intercept (a): {model.intercept_}")
print(f"Coefficients (b): {model.coef_}")

Intercept (a): 643.606246169947
Coefficients (b): [ 1.53847404e-03  1.15085625e-08  4.77163114e-06  3.81636842e-02
  3.19874990e-02  4.39983891e-01 -2.45075523e-01 -9.35059435e+00
 -9.08287015e+00  1.14768263e+01  6.95663794e+00 -1.54693469e+00
 -7.44829307e+00  8.99522705e+00  2.70816400e-07  9.86047579e+00
 -4.51179565e+01 -3.29271406e+01  6.81846192e+01 -4.92318894e+01
 -4.17305658e+02  4.66537548e+02 -7.09546567e+02  1.12280183e+02
 -1.70487286e+02 -5.08200654e+01  1.43771055e+02 -1.06299184e+02
  1.64436975e+03 -9.03307740e-01 -1.53107708e+02 -4.21288858e+02
 -9.17743540e+01  2.69129279e+02 -1.30597271e+02 -7.37414910e+02
  1.49124527e+02 -8.10203790e+01  3.34585097e+02 -2.12664948e+02
 -5.90647186e+00 -1.35280127e+01 -1.01789998e+02  1.64196150e+02
 -9.61197639e+01 -1.38187649e+02  5.22943826e+01 -1.01052875e+02
  1.69366993e+02 -1.45320977e+02 -6.80448266e+01 -1.02851974e+02
  2.94996872e+02 -1.01151751e+02  2.41002029e+01 -4.97006291e+00
 -3.39449434e+02 -6.58227850e+01 -1.0043

In [14]:
y_pred = model.predict(X_test)
y_pred

array([136.78640136, 102.49634123, 361.30588766, ..., -82.84204913,
       338.54031509, 283.26673883], shape=(1960,))

In [15]:
from sklearn.metrics import mean_squared_error as mse , r2_score

print(f"Mean squared error: {mse(y_test, y_pred)}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred)}")

Mean squared error: 511915.7643419609
Coefficient of determination: 0.23411742056680152
